In [1]:
import sys
# Append root path 
sys.path.append("../")
sys.path.append("../lmmnn")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

if tf.test.gpu_device_name() != '/device:GPU:0':
    print('WARNING: GPU device not found.')
else:
    print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

from model.mixed_effects import *
from utils.fe_models import get_model
from utils.evaluation import *
from utils.utils import *
from data.preprocessing import dataset_preprocessing

# from vis.utils.utils import apply_modifications
# helper function
def update_layer_activation(model, activation, index=-1):
    model.layers[index].activation = activation
    return model

from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Reshape, Embedding, Concatenate
from tensorflow.keras.activations import sigmoid

from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import roc_auc_score as auroc
from sklearn.metrics import f1_score as f1
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from tensorflow_addons.metrics import F1Score

from scipy import stats
import pickle
import yaml
import time
import gc

RS = 555

SUCCESS: Found GPU: /device:GPU:0


#### Download and save data from Pargent et al. by running "data/download_pargent2022_datasets.py before running this notebook

In [2]:
mode="cv"
hct=10
test_ratio=None
val_ratio=None
folds=5
results = {}
dataset_names = ["eucalyptus", "Midwest_survey", "hpc-job-scheduling", "video-game-sales", "okcupid-stem", "Diabetes130US"]


loss_use = lambda: tf.keras.losses.CategoricalCrossentropy
target= "categorical"
batch_size=512
epochs = 500
early_stopping = 20
model_name = "AutoGluon"
embed_dims_method = "AutoGluon"

results = {}

#######################################

for dataset_name in dataset_names:
    print(f"Start training procedure for {dataset_name}")
    data_path = f"{mode}_RS{RS}_hct{hct}"
    if mode == "cv":
        data_path += f"_{folds}folds"
    elif mode == "train_test":
        data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
    elif mode == "train_val_test":
        data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"

    # If no data_dict exists, run preprocessing, else load data_dict
    if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
        dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
    with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
            data_dict = pickle.load(handle)

    z_cols = data_dict["z_cols"]
    
    results[dataset_name] = {}
    for fold_num in range(folds):
        results[dataset_name][fold_num] = {}

        print(f"Fold no. {fold_num}")
        results[dataset_name][fold_num]["histories"] = {}
        results[dataset_name][fold_num]["predictions"] = {}
        results[dataset_name][fold_num]["times"] = {}
        results[dataset_name][fold_num]["other_info"] = {}
        results[dataset_name][fold_num] = {}

        print(f"Fold no. {fold_num}")
        results[dataset_name][fold_num]["histories"] = {}
        results[dataset_name][fold_num]["predictions"] = {}
        results[dataset_name][fold_num]["times"] = {}
        results[dataset_name][fold_num]["other_info"] = {}
        for K,R in zip([1,2,2,5],[0,0,1,1]):
            if K==2 and R==1:
                save_path = f"../results/{dataset_name}/{data_path}/fold_{fold_num}"
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
            else:
                save_path = f"../results/{dataset_name}/{data_path}/fold_{fold_num}/K_"+str(K)+"_R_"+str(R)
                if not os.path.exists(save_path):
                    os.makedirs(save_path)

        
            z_ohe_encoded_train = data_dict[f"z_ohe_encoded_train_{fold_num}"] 
            z_ohe_encoded_val = data_dict[f"z_ohe_encoded_val_{fold_num}"] 
            z_ohe_encoded_test = data_dict[f"z_ohe_encoded_test_{fold_num}"] 

            z_target_encoded_train = data_dict[f"z_target_encoded_train_{fold_num}"] 
            z_target_encoded_val = data_dict[f"z_target_encoded_val_{fold_num}"] 
            z_target_encoded_test = data_dict[f"z_target_encoded_test_{fold_num}"] 

            target_encoding_time = data_dict[f"target_encoding_time_{fold_num}"]
            ohe_encoding_time = data_dict[f"ohe_encoding_time_{fold_num}"]

            x_cols = data_dict[f"X_train_{fold_num}"].columns
            X_train = data_dict[f"X_train_{fold_num}"]
            Z_train = data_dict[f"Z_train_{fold_num}"]
            y_train = data_dict[f"y_train_{fold_num}"]

            X_val = data_dict[f"X_val_{fold_num}"]
            Z_val = data_dict[f"Z_val_{fold_num}"]
            y_val = data_dict[f"y_val_{fold_num}"]

            X_test = data_dict[f"X_test_{fold_num}"]
            Z_test = data_dict[f"Z_test_{fold_num}"]
            y_test = data_dict[f"y_test_{fold_num}"]

            if not os.path.exists(f"{save_path}/results_RS{RS}_{dataset_name}_iter{fold_num}.pickle"):

                tf.random.set_seed(RS+fold_num)
                np.random.seed(RS+fold_num)

                qs = np.max([tf.reduce_max(Z_train, axis=0),tf.reduce_max(Z_val, axis=0),tf.reduce_max(Z_test, axis=0)],axis=0)+1

                X_train = tf.convert_to_tensor(X_train)
                Z_train = tf.convert_to_tensor(Z_train,dtype=tf.int32)
                y_train = tf.convert_to_tensor(y_train)

                X_val = tf.convert_to_tensor(X_val)
                Z_val = tf.convert_to_tensor(Z_val,dtype=tf.int32)
                y_val = tf.convert_to_tensor(y_val)

                X_test = tf.convert_to_tensor(X_test)
                Z_test = tf.convert_to_tensor(Z_test,dtype=tf.int32)
                y_test = tf.convert_to_tensor(y_test)

                if target == "categorical":
                    n_classes = np.unique(y_train).shape[0]
                elif target=="binary":
                    n_classes = 1

                y_train = tf.one_hot(tf.cast(y_train,tf.int32),n_classes)
                y_val = tf.one_hot(tf.cast(y_val,tf.int32),n_classes)
                y_test = tf.one_hot(tf.cast(y_test,tf.int32),n_classes)

                ##### GMENN #####
                d = X_train.shape[1] # columns
                n = X_train.shape[0] # rows
                num_outputs = n_classes
                perc_numeric = d/(d+Z_train.shape[1])

    #             qs = np.max([tf.reduce_max(Z_train, axis=0),tf.reduce_max(Z_val, axis=0),tf.reduce_max(Z_test, axis=0)],axis=0)+1

                set_seed(RS)

                fe_model, optimizer = get_model(model_name=model_name, input_size=X_train.shape[1], 
                                                  output_size=num_outputs, 
                                                  target=target, 
                                                  perc_numeric=perc_numeric, RS=RS)
                
                if dataset_name=="eucalyptus":
                    optimizer.learning_rate.assign(optimizer.learning_rate*10)

                initial_stds = np.ones([len(qs),num_outputs]).astype(float).tolist()

                me_model = MixedEffectsNetwork(X_train, Z_train, y_train, fe_model, 
                                               target=target, qs=qs,
                                               initial_stds=initial_stds,
                                              fe_loss_weight=1.,
                                               mode="intercepts",
                                               early_stopping_fe=early_stopping,
                                              )    

                me_model.compile(
                    loss_class_me = loss_use()(),
                    loss_class_fe = loss_use()(),
                #     metric_class_me = tf.keras.metrics.AUC(multi_label=True, name="auc_me"),
                #     metric_class_fe = tf.keras.metrics.AUC(multi_label=True, name="auc_fe"),
                    optimizer=optimizer
                )

                mcmc = MCMCSamplingCallback(num_mcmc_samples=K,
                                            perc_burnin=0.7,
                                            warm_restart=None,
                                            num_burnin_steps=R,
                                            step_size = 0.1#initial_step_size,
                                       )


                print_metric = PrintMetrics(X_train, Z_train, y_train, X_val, Z_val, y_val)

                start = time.time()
                history = me_model.fit([X_train,Z_train], y_train,
                             callbacks=[mcmc,
                                        print_metric,
                                        tf.keras.callbacks.EarlyStopping(monitor="me_auc_val", patience=early_stopping, mode="max")],
                             epochs=epochs,
                             validation_data=[[X_val,Z_val],y_val],
                            batch_size=batch_size)

                end = time.time()
                fit_time_gmenn = round(end-start,2)

                y_train_pred_gmenn, y_train_pred_gmenn_fe = me_model([X_train,Z_train])
                y_val_pred_gmenn, y_val_pred_gmenn_fe = me_model([X_val,Z_val])
                y_test_pred_gmenn, y_test_pred_gmenn_fe = me_model([X_test,Z_test])    


                ###### Prepare NN Training ######



                ##### Document Results #####

                results[dataset_name][fold_num]["histories"]["GMENN_K"+str(K)+"_R"+str(R)] = history.history

                results[dataset_name][fold_num]["predictions"]["GMENN_K"+str(K)+"_R"+str(R)] = [y_train_pred_gmenn, y_val_pred_gmenn, y_test_pred_gmenn]
                
                results[dataset_name][fold_num]["times"]["GMENN_K"+str(K)+"_R"+str(R)] = fit_time_gmenn

                results[dataset_name][fold_num]["other_info"]["GMENN_K"+str(K)+"_R"+str(R)] = {
                        "_stddev_z": np.array([i.numpy() for i in me_model.data_model._stddev_z]),
                        "acceptance_rates": np.array(me_model.acceptance_rates),
                        "random_effects": me_model.mean_samples,
                        "all_samples": me_model.all_samples,
                        "stds": me_model.stds
                    }

                with open(f"{save_path}//results_RS{RS}_{dataset_name}_iter{fold_num}.pickle", 'wb') as handle:
                    pickle.dump(results[dataset_name][fold_num], handle, protocol=pickle.HIGHEST_PROTOCOL)


                del X_train, X_val, X_test, y_train, y_val, y_test

                gc.collect()
            else:
                if K==2 and R==1.:
                    print(f"Load results for dataset {dataset_name}, iteration={fold_num}")
                    with open(f"{save_path}/results_RS{RS}_{dataset_name}_iter{fold_num}.pickle", 'rb') as handle:
                        res = pickle.load(handle)
                    results[dataset_name][fold_num]["histories"]["GMENN_K"+str(K)+"_R"+str(R)] = res["histories"]["GMENN"]
                    results[dataset_name][fold_num]["predictions"]["GMENN_K"+str(K)+"_R"+str(R)] = res["predictions"]["GMENN"]
                    results[dataset_name][fold_num]["times"]["GMENN_K"+str(K)+"_R"+str(R)] = res["times"]["GMENN"]
                    results[dataset_name][fold_num]["other_info"]["GMENN_K"+str(K)+"_R"+str(R)] = res["other_info"]["GMENN"]
                    
                else:
                    print(f"Load results for dataset {dataset_name}, iteration={fold_num}")
                    with open(f"{save_path}/results_RS{RS}_{dataset_name}_iter{fold_num}.pickle", 'rb') as handle:
                        res = pickle.load(handle)
                    results[dataset_name][fold_num]["histories"]["GMENN_K"+str(K)+"_R"+str(R)] = res["histories"]["GMENN_K"+str(K)+"_R"+str(R)]
                    results[dataset_name][fold_num]["predictions"]["GMENN_K"+str(K)+"_R"+str(R)] = res["predictions"]["GMENN_K"+str(K)+"_R"+str(R)]
                    results[dataset_name][fold_num]["times"]["GMENN_K"+str(K)+"_R"+str(R)] = res["times"]["GMENN_K"+str(K)+"_R"+str(R)]
                    results[dataset_name][fold_num]["other_info"]["GMENN_K"+str(K)+"_R"+str(R)] = res["other_info"]["GMENN_K"+str(K)+"_R"+str(R)]


Start training procedure for eucalyptus
Fold no. 0
Fold no. 0
Random seed set as 555


2024-01-22 08:34:39.816861: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory



 Start sampling for epoch 1 of training
Epoch 1/500
1/1 [==============================] - 2s 2s/step - me_loss: 1.4721 - me_loss_val: 1.6373 - fe_loss: 1.4676 - fe_loss_val: 1.4793 - me_auc: 0.7271 - me_auc_val: 0.6758 - fe_auc: 0.7469 - fe_auc_val: 0.7200 - stds: 0.9291 - acceptance_rate: 1.0000

 Start sampling for epoch 2 of training
Epoch 2/500
1/1 [==============================] - 0s 333ms/step - me_loss: 1.0387 - me_loss_val: 1.2607 - fe_loss: 1.3474 - fe_loss_val: 1.3843 - me_auc: 0.8576 - me_auc_val: 0.7688 - fe_auc: 0.8249 - fe_auc_val: 0.7936 - stds: 0.8978 - acceptance_rate: 1.0000

 Start sampling for epoch 3 of training
Epoch 3/500
1/1 [==============================] - 0s 319ms/step - me_loss: 0.9327 - me_loss_val: 1.1547 - fe_loss: 1.2644 - fe_loss_val: 1.3183 - me_auc: 0.8868 - me_auc_val: 0.8070 - fe_auc: 0.8584 - fe_auc_val: 0.8294 - stds: 0.9116 - acceptance_rate: 0.9979

 Start sampling for epoch 4 of training
Epoch 4/500
1/1 [==============================] - 0s


 Start sampling for epoch 24 of training
Epoch 24/500
1/1 [==============================] - 0s 308ms/step - me_loss: 0.7133 - me_loss_val: 0.9395 - fe_loss: 0.8715 - fe_loss_val: 1.0213 - me_auc: 0.9560 - me_auc_val: 0.8782 - fe_auc: 0.9479 - fe_auc_val: 0.8798 - stds: 0.2834 - acceptance_rate: 0.2371

 Start sampling for epoch 25 of training
Epoch 25/500
1/1 [==============================] - 0s 347ms/step - me_loss: 0.7077 - me_loss_val: 0.9346 - fe_loss: 0.8608 - fe_loss_val: 1.0114 - me_auc: 0.9567 - me_auc_val: 0.8787 - fe_auc: 0.9485 - fe_auc_val: 0.8820 - stds: 0.2873 - acceptance_rate: 0.1523

 Start sampling for epoch 26 of training
Epoch 26/500
1/1 [==============================] - 0s 335ms/step - me_loss: 0.7029 - me_loss_val: 0.9362 - fe_loss: 0.8507 - fe_loss_val: 1.0022 - me_auc: 0.9587 - me_auc_val: 0.8768 - fe_auc: 0.9495 - fe_auc_val: 0.8832 - stds: 0.2736 - acceptance_rate: 0.2221

 Start sampling for epoch 27 of training
Epoch 27/500
1/1 [=========================


 Start sampling for epoch 51 of training
Epoch 51/500
1/1 [==============================] - 0s 374ms/step - me_loss: 0.6168 - me_loss_val: 0.8295 - fe_loss: 0.6801 - fe_loss_val: 0.8726 - me_auc: 0.9700 - me_auc_val: 0.8975 - fe_auc: 0.9623 - fe_auc_val: 0.8922 - stds: 0.1429 - acceptance_rate: 0.0057

 Start sampling for epoch 52 of training
Epoch 52/500
1/1 [==============================] - 0s 367ms/step - me_loss: 0.6135 - me_loss_val: 0.8286 - fe_loss: 0.6750 - fe_loss_val: 0.8692 - me_auc: 0.9700 - me_auc_val: 0.8971 - fe_auc: 0.9628 - fe_auc_val: 0.8931 - stds: 0.1418 - acceptance_rate: 0.0451

 Start sampling for epoch 53 of training
Epoch 53/500
1/1 [==============================] - 0s 368ms/step - me_loss: 0.6090 - me_loss_val: 0.8248 - fe_loss: 0.6698 - fe_loss_val: 0.8662 - me_auc: 0.9706 - me_auc_val: 0.8976 - fe_auc: 0.9635 - fe_auc_val: 0.8935 - stds: 0.1439 - acceptance_rate: 0.1003

 Start sampling for epoch 54 of training
Epoch 54/500
1/1 [=========================

...

120/120 [==============================] - 1s 8ms/step - me_loss: 0.7870 - me_loss_val: 0.9583 - fe_loss: 0.8473 - fe_loss_val: 0.9943 - me_auc: 0.7637 - me_auc_val: 0.6509 - fe_auc: 0.7166 - fe_auc_val: 0.6171 - stds: 0.2961 - acceptance_rate: 0.8613

 Start sampling for epoch 24 of training
Epoch 24/500
120/120 [==============================] - 1s 9ms/step - me_loss: 0.7750 - me_loss_val: 0.9560 - fe_loss: 0.8364 - fe_loss_val: 0.9940 - me_auc: 0.7721 - me_auc_val: 0.6501 - fe_auc: 0.7253 - fe_auc_val: 0.6154 - stds: 0.2889 - acceptance_rate: 0.9987

 Start sampling for epoch 25 of training
Epoch 25/500
111/120 [==========================>...] - ETA: 0s
 Early stopping of FE by fe_auc_val at 25 epochs
120/120 [==============================] - 1s 9ms/step - me_loss: 0.7767 - me_loss_val: 0.9669 - fe_loss: 0.8393 - fe_loss_val: 1.0078 - me_auc: 0.7713 - me_auc_val: 0.6491 - fe_auc: 0.7251 - fe_auc_val: 0.6145 - stds: 0.2847 - acceptance_rate: 0.9987

 Start sampling for epoch 26 of t


 Start sampling for epoch 15 of training
Epoch 15/500
120/120 [==============================] - 1s 8ms/step - me_loss: 0.8439 - me_loss_val: 0.9525 - fe_loss: 0.8869 - fe_loss_val: 0.9667 - me_auc: 0.7238 - me_auc_val: 0.6474 - fe_auc: 0.6830 - fe_auc_val: 0.6217 - stds: 0.3343 - acceptance_rate: 0.9393

 Start sampling for epoch 16 of training
Epoch 16/500
120/120 [==============================] - 1s 8ms/step - me_loss: 0.8400 - me_loss_val: 0.9558 - fe_loss: 0.8930 - fe_loss_val: 0.9807 - me_auc: 0.7280 - me_auc_val: 0.6499 - fe_auc: 0.6842 - fe_auc_val: 0.6225 - stds: 0.3259 - acceptance_rate: 0.9338

 Start sampling for epoch 17 of training
Epoch 17/500
120/120 [==============================] - 1s 7ms/step - me_loss: 0.8265 - me_loss_val: 0.9550 - fe_loss: 0.8837 - fe_loss_val: 0.9843 - me_auc: 0.7350 - me_auc_val: 0.6431 - fe_auc: 0.6871 - fe_auc_val: 0.6126 - stds: 0.3140 - acceptance_rate: 0.9443

 Start sampling for epoch 18 of training
Epoch 18/500
120/120 [===============

Adapt step size to 0.02500000037252903

 Start sampling for epoch 3 of training
Epoch 3/500
120/120 [==============================] - 1s 12ms/step - me_loss: 2.0771 - me_loss_val: 2.0977 - fe_loss: 1.0252 - fe_loss_val: 1.0271 - me_auc: 0.5436 - me_auc_val: 0.5372 - fe_auc: 0.6383 - fe_auc_val: 0.6270 - stds: 0.9986 - acceptance_rate: 0.0000e+00
Adapt step size to 0.012500000186264515

 Start sampling for epoch 4 of training
Epoch 4/500
120/120 [==============================] - 1s 8ms/step - me_loss: 1.9732 - me_loss_val: 2.0012 - fe_loss: 0.9726 - fe_loss_val: 0.9843 - me_auc: 0.5528 - me_auc_val: 0.5448 - fe_auc: 0.6456 - fe_auc_val: 0.6286 - stds: 0.9852 - acceptance_rate: 0.6667

 Start sampling for epoch 5 of training
Epoch 5/500
120/120 [==============================] - 1s 7ms/step - me_loss: 1.7374 - me_loss_val: 1.7726 - fe_loss: 0.9959 - fe_loss_val: 1.0118 - me_auc: 0.5652 - me_auc_val: 0.5552 - fe_auc: 0.6462 - fe_auc_val: 0.6269 - stds: 0.9561 - acceptance_rate: 1.0000




 Start sampling for epoch 18 of training
Epoch 18/500
120/120 [==============================] - 1s 8ms/step - me_loss: 0.8365 - me_loss_val: 0.9874 - fe_loss: 0.9718 - fe_loss_val: 1.0804 - me_auc: 0.7371 - me_auc_val: 0.6455 - fe_auc: 0.6857 - fe_auc_val: 0.6178 - stds: 0.4018 - acceptance_rate: 1.0000

 Start sampling for epoch 19 of training
Epoch 19/500
120/120 [==============================] - 1s 7ms/step - me_loss: 0.8124 - me_loss_val: 0.9761 - fe_loss: 0.9404 - fe_loss_val: 1.0622 - me_auc: 0.7498 - me_auc_val: 0.6460 - fe_auc: 0.6989 - fe_auc_val: 0.6184 - stds: 0.4074 - acceptance_rate: 1.0000

 Start sampling for epoch 20 of training
Epoch 20/500
120/120 [==============================] - 1s 7ms/step - me_loss: 0.8196 - me_loss_val: 0.9965 - fe_loss: 0.9436 - fe_loss_val: 1.0801 - me_auc: 0.7490 - me_auc_val: 0.6441 - fe_auc: 0.6978 - fe_auc_val: 0.6151 - stds: 0.3946 - acceptance_rate: 0.9803

 Start sampling for epoch 21 of training
Epoch 21/500
120/120 [===============


 Start sampling for epoch 7 of training
Epoch 7/500
120/120 [==============================] - 1s 7ms/step - me_loss: 1.2210 - me_loss_val: 1.2785 - fe_loss: 0.9884 - fe_loss_val: 1.0141 - me_auc: 0.6172 - me_auc_val: 0.5935 - fe_auc: 0.6552 - fe_auc_val: 0.6295 - stds: 0.6169 - acceptance_rate: 0.9310

 Start sampling for epoch 8 of training
Epoch 8/500
120/120 [==============================] - 1s 8ms/step - me_loss: 1.1055 - me_loss_val: 1.1719 - fe_loss: 1.0223 - fe_loss_val: 1.0537 - me_auc: 0.6389 - me_auc_val: 0.6086 - fe_auc: 0.6585 - fe_auc_val: 0.6311 - stds: 0.5454 - acceptance_rate: 0.9964

 Start sampling for epoch 9 of training
Epoch 9/500
120/120 [==============================] - 1s 8ms/step - me_loss: 1.0309 - me_loss_val: 1.1053 - fe_loss: 1.0291 - fe_loss_val: 1.0646 - me_auc: 0.6564 - me_auc_val: 0.6192 - fe_auc: 0.6584 - fe_auc_val: 0.6277 - stds: 0.5176 - acceptance_rate: 0.9962

 Start sampling for epoch 10 of training
Epoch 10/500
120/120 [=====================

## Evaluation

### Performance

In [8]:
models = ["GMENN_K"+str(K)+"_R"+str(R) for K,R in zip([1,2,2,5],[0,0,1,1])]

results_perf = {dataset_name: {num: {model: {}  for model in models} for num in range(folds)} for dataset_name in dataset_names}
for dataset_name in dataset_names:
    try:
        with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
            data_dict = pickle.load(handle)        
    except:
        print(f"dataset {dataset_name} not found") 
    for num in range(folds):
        y_test = data_dict[f"y_test_{num}"]
        n_classes = np.unique(y_test).shape[0]
        y_test = tf.one_hot(data_dict[f"y_test_{num}"],n_classes)
        for model in models:
            try:
                y_pred = results[dataset_name][num]["predictions"][model][2]

                results_perf[dataset_name][num][model] = get_metrics(y_test,y_pred,target)
                results_perf[dataset_name][num][model]["Time"] = results[dataset_name][num]["times"][model]
            except:
                print(f"Set nan for {dataset_name}, {num}")
                results_perf[dataset_name][num][model] = {"Accuracy": np.nan,
                                                          "AUROC": np.nan,
                                                          "F1": np.nan,
                                                          "Time": np.nan}
#                 print(f"Didnt work for {dataset_name}, {num}")


In [9]:
metric = "AUROC"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

round_mean_at = 2
round_std_at = 3

for dataset_name in dataset_names:
    dataset_models = list(results_perf[dataset_name][0].keys())
    use_df = pd.DataFrame([pd.DataFrame(results_perf[dataset_name][fold_num]).loc[metric,models] for fold_num in results_perf[dataset_name].keys()],index=results_perf[dataset_name].keys())
    
    df_mean = pd.DataFrame(use_df.mean(axis=0).round(round_mean_at).astype(str) + " (" + use_df.std(axis=0).round(round_std_at).astype(str) + ")").transpose()
    model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}
    dataset_res_dict[dataset_name] = model_dict
    
    best_models[dataset_name] = use_df.columns[use_df.mean(axis=0).argmax()]

    t_test_res = np.array([stats.ttest_rel(use_df[best_models[dataset_name]].values, use_df[model].values)[1] if model in dataset_models else 0 for model in models]).round(3)
    t_test_res[np.isnan(t_test_res)] = 1.
    t_test_results[dataset_name] = t_test_res
    
res_df = pd.DataFrame(dataset_res_dict).transpose()
    
def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_results[dataset_name][i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

# res_df.style.apply(negative_bold)
res_df

,GMENN_K1_R0,GMENN_K2_R0,GMENN_K2_R1,GMENN_K5_R1
eucalyptus,0.89 (0.026),0.9 (0.025),0.9 (0.022),0.9 (0.029)
Midwest_survey,0.89 (0.007),0.88 (0.005),0.88 (0.023),0.89 (0.006)
hpc-job-scheduling,0.91 (0.007),0.9 (0.008),0.91 (0.008),0.91 (0.008)
video-game-sales,0.78 (0.01),0.79 (0.009),0.79 (0.009),0.79 (0.009)
okcupid-stem,0.8 (0.003),0.8 (0.004),0.8 (0.004),0.8 (0.003)
Diabetes130US,0.64 (0.004),0.65 (0.001),0.65 (0.002),0.64 (0.005)


In [12]:
res_df["GMENN_K1_R0"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN_K2_R0"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN_K2_R1"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN_K5_R1"].apply(lambda x: float(x.split(" ")[0])).values

(array([0.89, 0.89, 0.91, 0.78, 0.8 , 0.64]),
 array([0.9 , 0.88, 0.9 , 0.79, 0.8 , 0.65]),
 array([0.9 , 0.88, 0.91, 0.79, 0.8 , 0.65]),
 array([0.9 , 0.89, 0.91, 0.79, 0.8 , 0.64]))

### Time

In [13]:
metric = "Time"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

round_mean_at = 2
round_std_at = 3

for dataset_name in dataset_names:
    dataset_models = list(results_perf[dataset_name][0].keys())
    use_df = pd.DataFrame([pd.DataFrame(results_perf[dataset_name][fold_num]).loc[metric,models] for fold_num in results_perf[dataset_name].keys()],index=results_perf[dataset_name].keys())/60
    
    df_mean = pd.DataFrame(use_df.mean(axis=0).round(round_mean_at).astype(str) + " (" + use_df.std(axis=0).round(round_std_at).astype(str) + ")").transpose()
    model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}
    dataset_res_dict[dataset_name] = model_dict
    
    best_models[dataset_name] = use_df.columns[use_df.mean(axis=0).argmin()]

    t_test_res = np.array([stats.ttest_rel(use_df[best_models[dataset_name]].values, use_df[model].values)[1] if model in dataset_models else 0 for model in models]).round(3)
    t_test_res[np.isnan(t_test_res)] = 1.
    t_test_results[dataset_name] = t_test_res
    
res_df = pd.DataFrame(dataset_res_dict).transpose()
    
def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_results[dataset_name][i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

# res_df.style.apply(negative_bold)
res_df

,GMENN_K1_R0,GMENN_K2_R0,GMENN_K2_R1,GMENN_K5_R1
eucalyptus,0.9 (0.548),3.66 (3.317),10.32 (14.13),13.53 (8.814)
Midwest_survey,0.88 (0.106),1.2 (0.109),2.3 (0.828),2.5 (0.476)
hpc-job-scheduling,2.02 (0.84),3.2 (1.761),5.7 (1.43),18.36 (8.095)
video-game-sales,1.41 (0.463),1.85 (0.521),2.29 (0.399),3.25 (1.253)
okcupid-stem,1.44 (0.28),2.66 (0.432),4.48 (0.366),10.72 (4.371)
Diabetes130US,1.79 (0.138),2.59 (0.154),5.06 (0.313),6.71 (0.427)


In [14]:
res_df["GMENN_K1_R0"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN_K2_R0"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN_K2_R1"].apply(lambda x: float(x.split(" ")[0])).values,res_df["GMENN_K5_R1"].apply(lambda x: float(x.split(" ")[0])).values

(array([0.9 , 0.88, 2.02, 1.41, 1.44, 1.79]),
 array([3.66, 1.2 , 3.2 , 1.85, 2.66, 2.59]),
 array([10.32,  2.3 ,  5.7 ,  2.29,  4.48,  5.06]),
 array([13.53,  2.5 , 18.36,  3.25, 10.72,  6.71]))